<a href="https://colab.research.google.com/github/somwrks/AI-Solar-Panel/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self Rotatory Solar Panel

This Project utilizes various vision models to train on planet sun labelled datasets and perform validation checks to determine which model performs the best and converts the best model to tinyML Model to make it usable by microcontroller

## Installing Dependencies

In [1]:
%pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
%pip install  tensorflow[and-cuda] tensorflow-datasets roboflow ultralytics torch torchvision matplotlib tqdm


## Importing Libraries

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
from tensorflow.keras import mixed_precision
from roboflow import Roboflow
import yaml
import os
from PIL import Image


In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['NO_ALBUMENTATIONS_UPDATE '] = '1'


In [5]:
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices())


TensorFlow version: 2.17.1
GPU Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
# List available GPUs
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

if len(physical_devices) > 0:
# Enable memory growth for all GPUs
  for gpu in physical_devices:
      tf.config.experimental.set_memory_growth(gpu, True)

  # Use all available GPUs
  tf.config.set_visible_devices(physical_devices, 'GPU')

  # Enable XLA (Accelerated Linear Algebra) optimization
  tf.config.optimizer.set_jit(True)

  tf.config.set_visible_devices(physical_devices[0], 'GPU')

  # Enable mixed precision training

  policy = mixed_precision.Policy('mixed_float16')
  mixed_precision.set_global_policy(policy)

  print("GPU optimization settings applied successfully")
else:
    print("No GPU available. Running on CPU.")

Num GPUs Available:  1
GPU optimization settings applied successfully


## Loading Datasets

In [7]:
import os
import yaml
from roboflow import Roboflow
from PIL import Image

# Download both datasets
rf = Roboflow(api_key="")

# Dataset 1

project1 = rf.workspace("yassine-pzpt7").project("sun-tracking-photovoltaic-panel")
version1 = project1.version(2)
dataset1 = version1.download("yolov8")

# Dataset 2
project2 = rf.workspace("1009727588-qq-com").project("sun-nxvfz")
dataset2 = project2.version(2).download("yolov8")

# Dataset 3
project3 = rf.workspace("rik-tjduw").project("sun-tracking-555mn")
version3 = project3.version(4)
dataset3 = version3.download("yolov8")

# Dataset 4
project4 = rf.workspace("stardetect").project("solar-re1fe")
version4 = project4.version(1)
dataset4 = version4.download("yolov8")

# Dataset 5
project5 = rf.workspace("fruitdetection-ulcz9").project("sun_detection-hl04q")
version5 = project5.version(1)
dataset5 = version5.download("yolov8")

# Prepare dataset paths
dataset1_path = dataset1.location
dataset2_path = dataset2.location
dataset3_path = dataset3.location
dataset4_path = dataset4.location
dataset5_path = dataset5.location

def resize_images(directory, target_resolution=(640, 640)):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(image_extensions):
                image_path = os.path.join(root, file)
                try:
                    with Image.open(image_path) as img:
                        img_resized = img.resize(target_resolution, Image.LANCZOS)
                        img_resized.save(image_path)
                except Exception as e:
                    print(f"Error resizing image {image_path}: {e}")

# Function to count images in a directory
def count_images(directory):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    return len([f for f in os.listdir(directory) if f.lower().endswith(image_extensions)])

# Function to combine datasets

def combine_datasets(*dataset_paths, combined_path):
    for split in ['train', 'valid', 'test']:
        os.makedirs(os.path.join(combined_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(combined_path, split, 'labels'), exist_ok=True)

        for dataset_path in dataset_paths:
            images_src = os.path.join(dataset_path, split, 'images')
            labels_src = os.path.join(dataset_path, split, 'labels')

            images_dst = os.path.join(combined_path, split, 'images')
            labels_dst = os.path.join(combined_path, split, 'labels')

            if os.path.exists(images_src):
                for file in os.listdir(images_src):
                    shutil.copy2(os.path.join(images_src, file), images_dst)
            if os.path.exists(labels_src):
                for file in os.listdir(labels_src):
                    shutil.copy2(os.path.join(labels_src, file), labels_dst)


# Combine datasets
combined_path = './combined_dataset'
combine_datasets(dataset1_path, dataset2_path, dataset3_path,dataset4_path, dataset5_path, combined_path=combined_path)

# Resize images in the combined dataset
for split in ['train', 'valid', 'test']:
    images_dir = os.path.join(combined_path, split, 'images')
    resize_images(images_dir)

print("Images resized to target resolution.")


# Count images in each dataset
train_count = len(os.listdir(os.path.join(combined_path, 'train', 'images')))
valid_count = len(os.listdir(os.path.join(combined_path, 'valid', 'images')))
test_count = len(os.listdir(os.path.join(combined_path, 'test', 'images')))
num_classes = len(set([f.split('_')[0] for f in os.listdir(os.path.join(combined_path, 'train', 'labels'))]))

print(f"Number of images in training set: {train_count}")
print(f"Number of images in validation set: {valid_count}")
print(f"Number of images in test set: {test_count}")
print(f"Total number of images: {train_count + valid_count + test_count}")

# Create YAML file for YOLOv5
yaml_content = {
    'train': os.path.join(combined_path, 'train'),
    'val': os.path.join(combined_path, 'valid'),
    'test': os.path.join(combined_path, 'test'),
    'nc': len(os.listdir(os.path.join(combined_path, 'train', 'labels'))),
    'names': [f'class_{i}' for i in range(len(os.listdir(os.path.join(combined_path, 'train', 'labels'))))],
    'image_counts': {
        'train': train_count,
        'val': valid_count,
        'test': test_count
    }
}


yaml_content = {
    'train': os.path.join(combined_path, 'train'),
    'val': os.path.join(combined_path, 'valid'),
    'test': os.path.join(combined_path, 'test'),
    'nc': num_classes,
    'names': [f'class_{i}' for i in range(num_classes)],
}

yaml_path = os.path.join(combined_path, 'dataset.yaml')
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)

print(f"Dataset YAML file created at: {yaml_path}")


loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
loading Roboflow workspace...
loading Roboflow project...
Images resized to target resolution.
Number of images in training set: 3597
Number of images in validation set: 633
Number of images in test set: 443
Total number of images: 4673
Dataset YAML file created at: ./combined_dataset/dataset.yaml


## Setting Up Models


### 2. YoloV5 (LibRT)

In [8]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [10]:
from ultralytics import YOLO

yolo_model = YOLO('runs/detect/train6/weights/best.pt')

full_yaml_path = os.path.abspath(yaml_path)
yolo_results = yolo_model.train(
    data=full_yaml_path,
    epochs=1000,
    imgsz=640,
    lr0=0.001,  # Reduced initial learning rate for fine-tuning
    lrf=0.0001,  # Reduced final learning rate
    batch=-1,  # Let YOLO auto-determine the optimal batch size
    device=[0],
    augment=True,
    cos_lr=True,
    amp=True,  # Enable Automatic Mixed Precision
    resume=True,  # Resume training from the last checkpoint
    patience=300,  # Increase patience for early stopping
    mixup=0.2,  # Add mixup augmentation
    mosaic=1.0,  # Increase mosaic augmentation
    close_mosaic=10  # Disable mosaic augmentation for the last 10 epochs
)

# Evaluate the model after training
metrics = yolo_model.val()
print(f"mAP50-95: {metrics.box.map}")


Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/combined_dataset/dataset.yaml, epochs=500, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

train: Scanning /content/combined_dataset/train/labels.cache... 3597 images, 218 backgrounds, 0 corrupt: 100%|██████████| 3597/3597 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 579, len(boxes) = 3384. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA A100-SXM4-40GB) 39.56G total, 0.39G reserved, 0.37G allocated, 38.80G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


...

    208/500      26.3G     0.7859      0.519      1.121         49        640: 100%|██████████| 88/88 [00:34<00:00,  2.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.21it/s]

                   all        633        614      0.876      0.938      0.933      0.735
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 108, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



208 epochs completed in 2.289 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 88.2MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 88.2MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 268 layers, 43,871,832 parameters, 0 gradients, 166.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:06<00:00,  1.21it/s]


                   all        633        614      0.895      0.917       0.94      0.735
               class_0        292        292      0.788      0.798      0.845      0.404
               class_1        141        141      0.903      0.957      0.981      0.843
               class_2        181        181      0.994      0.994      0.995      0.959
Speed: 0.1ms preprocess, 6.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train6


In [11]:
# Evaluate YOLOv5
yolo_metrics = yolo_model.val()

print("YOLOv5 mAP:", yolo_metrics.results_dict['metrics/mAP50-95(B)'])


Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Model summary (fused): 268 layers, 43,871,832 parameters, 0 gradients, 166.3 GFLOPs


val: Scanning /content/combined_dataset/valid/labels.cache... 633 images, 19 backgrounds, 0 corrupt: 100%|██████████| 633/633 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:09<00:00,  1.72it/s]


                   all        633        614      0.892      0.917      0.939      0.735
               class_0        292        292      0.784      0.798      0.842      0.402
               class_1        141        141      0.897      0.957      0.981      0.845
               class_2        181        181      0.994      0.994      0.995      0.959
Speed: 0.1ms preprocess, 9.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train62
YOLOv5 mAP: 0.7352802519489792


## Testing Models

In [ ]:
# tbd

## Saving Model

In [ ]:
model.save('sun_tracker_model.h5')


sample_data/
